In [75]:
import json
import numpy as np
from pandas import DataFrame

# 영화 한줄평 데이터 불러오기
with open('train_data.json', encoding="utf-8") as data_file:    
    data = json.load(data_file)
data = DataFrame(data)

In [76]:
data.head()

,date,movie_id,rating,review
0,15.08.26,92575,4,종합 평점은 4점 드립니다.
1,15.08.25,92575,1,원작이 칭송받는 이유는 웹툰 계 자체의 질적 저하가 심각하기 때문. 원작이나 영화...
2,15.08.23,92575,10,나름의 감동도 있고 안타까운 마음에 가슴도 먹먹 배우들의 연기가 good 김수현...
3,15.08.23,92575,1,이런걸 돈주고 본 내자신이 후회스럽다 최악의 쓰레기 영화 김수현 밖에없는 저질 삼류영화
4,15.08.21,92575,7,"초반엔 코미디, 후반엔 액션, 결론은 코미디."


In [85]:
from sklearn.cross_validation import train_test_split 

train, test, y_train, y_test = train_test_split(data['review'], data['rating'], random_state = 0, test_size=0.1)

print(len(train), len(test))   
print(len(label_train), len(label_test)) 

630000 70000
630000 70000


In [9]:
import re
#text1 = [re.sub('\d+',' ',tmp) for tmp in train]
train_text = [re.sub('\W+',' ',tmp) for tmp in train]
#text1 = [re.sub('\d+',' ',tmp) for tmp in test]
test_text = [re.sub('\W+',' ',tmp) for tmp in test]

train_text

['올해 최고의 영화',
 '흥미롭고 시간 가는 줄 몰랐던 영화',
 '감수성이 풍부해서 중간중간 울컥울컥했슴 ㅠㅠ 최고임',
 '감동 깊게 봤습니다 자칫 소홀할수 있었던 탈북자 분들이나 소외되어 있는 분들의 문제를 영화화 했다는 것 자체 만으로도 모든 출연진 분들께 박수 쳐드리고 싶습니다 ',
 '김효진이 정말 매력적인 배우란걸 확실하게 각인 시켜줬다',
 'after all this time always ',
 'OOO기방송',
 '미국 코드가 안맞나 ',
 '평점이 왜 이모양인지 모르겠다',
 '테크노 액션 남들이 뭐라하든 난 연걸형님 팬이양 ',
 '스토리가 정직하게 흘러간다 그러면서도 긴장감있는 전개와 연기력이 좋았다',
 '몇년 지났는데 아직도 노래 생각남 ㅋㅋㅋ 넌 멋지잖아 넌 강하잖아 목소리도 정말 좋잖아 엄마랑 동생이랑 둘러 앉아 봤던 기억이 생생하네여',
 '이태임 때문에 봤었는데 ㅠㅠ',
 '이만하면 잘 만들지 않았나 반전도 괜찮았고 신선해서 좋았다 ',
 '재미없서용보지말라귱ㅇ',
 '난 누가범일일까 궁금해미쳐버리겠던데 잼나',
 '보고있는데 도시락장면에서 충격 15세맞나 확인하고옴 친구랑보다 캐민망 끝까지 보면 훨씩더충격일듯 ',
 '섬세한 연출 배우들 연기 색다른 복수 다 좋았다 ',
 '뻔한 설정에 실망스러운 결말 그냥 성의가 없다 ',
 '가끔씩 생각난다 모모세의 뒷모습이 ',
 '소름 돋네요 영화 곳곳의 많은 요소가 감동을 자아냅니다 교수의 행동 변화 젬베 이민자들 이 영화 추천합니다 ',
 '아름다운 풍경과 정성스럽게 내린 맛있는 커피 김이 모락모락 나는 방금 구운 빵 아름다운 마음을 가진 부부가 있는 곳 카페 마니 행복한 영화였어요 ',
 '마케팅도 최소한의 양심은 있게 해라 덕분에 교훈은 얻었다 다시는 이런 예술성을 가장한 쓰레기 영화 돈 주고 안본다 ',
 '정말5월을맞이해서가족들이함께보기에 딱 좋은 영화',
 'ㅋㅋ 저이거찍어씀 주인공친구역으로',
 '아름다운 영상미에 진지한 인생사가 스며들어있는 영화 ',
 '아이들이넘

In [52]:
f = open("train_pumsa.txt", 'r')
train_pumsa = f.read()
f.close()

In [53]:
f = open("test_pumsa.txt", 'r')
test_pumsa = f.read()
f.close()

In [57]:
def txt_preprocessing(text):
    k = text.split("],")
    kk = [txt.replace("[","") for txt in k]
    kk = [txt.replace("'","") for txt in kk]
    kk = [txt.replace(" ","") for txt in kk]
    kkk = [txt.split(",") for txt in kk]
    return kkk
train_pumsa = txt_preprocessing(train_pumsa)
test_pumsa = txt_preprocessing(test_pumsa)

In [61]:
train_pumsa[0]

['올해', '최고', '영화']

In [65]:
from gensim.models.keyedvectors import KeyedVectors
model_1 = KeyedVectors.load_word2vec_format('my.embedding1', binary=False, encoding='utf-8')

In [67]:
w2v_100 = dict(zip(model_1.wv.index2word, model_1.wv.syn0))
max_features = len(model_1.wv.vocab)
print(len(model_1.wv.vocab))

In [72]:
def seq_embedding(text):
    return np.array([w2v_100[w] for w in text if w in w2v_100])

train_emb = list(map(seq_embedding,train_pumsa))
test_emb = list(map(seq_embedding,test_pumsa))

In [73]:
# 문장 길이 맞춰주기(반복 패딩)
train_pd = sequence.pad_sequences(train_emb, maxlen=10, dtype='float64')
test_pd = sequence.pad_sequences(test_emb, maxlen=10, dtype='float64')

In [70]:
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, RNN, Flatten, Bidirectional
from keras.engine.topology import Input 
from keras.optimizers import Adagrad, SGD, Adam
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [86]:
model = Sequential()

#model.add(Embedding()) # input_dim / output_dim / input_length
model.add(GRU(128, input_shape=(10, 100)))
model.add(Dense(1))

ada = Adagrad(lr=0.01, decay=1e-6)
model.compile(loss='mean_squared_error', optimizer=ada)

print('Train...')
model.fit(train_pd, y_train, batch_size=128, epochs=1)

Train...
Epoch 1/1
630000/630000 [==============================] - 88s 140us/step - loss: 5.7400


In [87]:
score = model.evaluate(test_pd, y_test, batch_size=128)
score

70000/70000 [==============================] - 4s 54us/step


5.664241201673235

In [82]:
model = Sequential()

model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=(10, 100)))
model.add(Bidirectional(GRU(128)))
model.ad₩d(Dense(1))
ada = Adagrad(lr=0.01, decay=1e-6)
model.compile(loss='mean_squared_error', optimizer=ada)

print('Train...')
model.fit(train_pd, y_train, batch_size=128, epochs=1)

Train...
Epoch 1/1
630000/630000 [==============================] - 438s 696us/step - loss: 5.6159


In [83]:
score = model.evaluate(test_pd, y_test, batch_size=128)
score

70000/70000 [==============================] - 38s 542us/step


5.496863718196324